# Setting

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import matplotlib.font_manager as fm
import platform
plt.rcParams['font.family'] = 'AppleGothic'

# Preprocessing

## 서울시 건설 알림이 정보

In [61]:
construction_df = pd.read_csv('data/raw_data/서울시 건설 알림이 정보.csv', encoding='cp949')

In [62]:
construction_df.head()

,프로젝트 코드,프로젝트 명,사업착수일(계약일),사업기간,프로젝트 종료여부(진행:0 종료:1),사무실주소,프로젝트 주소,위치좌표(위도),위치좌표(경도),자치구 구분,사업금액(억원)
0,3502024051377,2024년 중부수도사업소 관내 포장도로 굴착복구공사(장기1차- 단가계약),202405,2024-05-13~2025-06-30,0,NaN,서울 중구 장충단로 88,37.566779,126.978618,NaN,NaN
1,7552024061894,"2024년 1,2호선 강북구간 궤도시설 보수보강공사",202406,2024-06-18~2025-06-24,0,NaN,서울 중구 정동 5-8,37.565345,126.977198,중구,NaN
2,8502024070466,의회청사 공간재배치 및 내진보강 소방공사 (장기1차),202407,2024-07-04~2025-06-09,0,NaN,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN
3,8502024061277,의회청사 공간재배치 및 내진보강 전기공사 (장기1차),202406,2024-06-12~2025-05-31,0,NaN,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN
4,8502024062180,의회청사 공간재배치 및 내진보강 정보통신공사 (장기1차),202406,2024-06-21~2025-05-31,0,NaN,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN


In [63]:
# 프로젝트 코드 컬럼 삭제 (의미 없음)
construction_df.drop(columns=["프로젝트 코드"], inplace=True)

In [64]:
# 사무실 주소 컬럼 삭제 (의미 없음)
construction_df.drop(columns=["사무실주소"], inplace=True)

In [65]:
# 프로젝트 종료여부(진행:0 종료:1)

# 컬럼 이름 변경
construction_df = construction_df.rename(columns={"프로젝트 종료여부(진행:0 종료:1)": "프로젝트 종료여부"})

# 0 → False, 1 → True 변환
construction_df["프로젝트 종료여부"] = construction_df["프로젝트 종료여부"].map({0: False, 1: True})

In [66]:
# 위도 경도 결측치 제거
construction_df.dropna(subset=["위치좌표(위도)", "위치좌표(경도)"], inplace=True)

In [67]:
# 자취구 컬럼 결측 보완
construction_df["자치구 구분"] = construction_df["자치구 구분"].fillna(
    construction_df["프로젝트 주소"].str.extract(r"\s([^\s]+구)")[0]
)

In [68]:
construction_df.isnull().sum()

프로젝트 명          0
사업착수일(계약일)      0
사업기간            0
프로젝트 종료여부       0
프로젝트 주소        12
위치좌표(위도)        0
위치좌표(경도)        0
자치구 구분         53
사업금액(억원)      537
dtype: int64

In [69]:
# '프로젝트 주소'와 '자치구 구분' 이렇게 했는데도 결측치가 있는 행 제거
construction_df.dropna(subset=["프로젝트 주소", "자치구 구분"], inplace=True)

In [70]:
# 사업착수일 int → datetime
construction_df["사업착수일"] = pd.to_datetime(construction_df["사업착수일(계약일)"].astype(str) + "01", format="%Y%m%d")

# 사업기간 → 시작일, 종료일 분리 및 datetime 변환
construction_df[["사업시작일", "사업종료일"]] = construction_df["사업기간"].str.split("~", expand=True)
construction_df["사업시작일"] = pd.to_datetime(construction_df["사업시작일"], errors="coerce")
construction_df["사업종료일"] = pd.to_datetime(construction_df["사업종료일"], errors="coerce")

# 기존 컬럼 삭제
construction_df.drop(columns=["사업착수일(계약일)"], inplace=True)
construction_df.drop(columns=["사업기간"], inplace=True)

In [71]:
construction_df.isnull().sum()

프로젝트 명         0
프로젝트 종료여부      0
프로젝트 주소        0
위치좌표(위도)       0
위치좌표(경도)       0
자치구 구분         0
사업금액(억원)     506
사업착수일          0
사업시작일          0
사업종료일          0
dtype: int64

In [72]:
construction_df.head()

,프로젝트 명,프로젝트 종료여부,프로젝트 주소,위치좌표(위도),위치좌표(경도),자치구 구분,사업금액(억원),사업착수일,사업시작일,사업종료일
0,2024년 중부수도사업소 관내 포장도로 굴착복구공사(장기1차- 단가계약),False,서울 중구 장충단로 88,37.566779,126.978618,중구,NaN,2024-05-01,2024-05-13,2025-06-30
1,"2024년 1,2호선 강북구간 궤도시설 보수보강공사",False,서울 중구 정동 5-8,37.565345,126.977198,중구,NaN,2024-06-01,2024-06-18,2025-06-24
2,의회청사 공간재배치 및 내진보강 소방공사 (장기1차),False,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN,2024-07-01,2024-07-04,2025-06-09
3,의회청사 공간재배치 및 내진보강 전기공사 (장기1차),False,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN,2024-06-01,2024-06-12,2025-05-31
4,의회청사 공간재배치 및 내진보강 정보통신공사 (장기1차),False,서울특별시 중구 세종대로 125,37.567728,126.976671,중구,NaN,2024-06-01,2024-06-21,2025-05-31


In [73]:
construction_df.to_csv("data/processed/construction_processed.csv", index=False, encoding="utf-8-sig")

## 서울시 건축물 유출지하수 현황

In [119]:
building_underground_water_df = pd.read_csv('data/raw_data/서울시 건축물 유출지하수 현황.csv', encoding='cp949')

In [120]:
building_underground_water_df.head()

,건축물명,건축물 위치,층수(지상_지하),연면적,조사년도,총발생량(톤),일평균발생량(톤/일),일평균이용현황(톤/일)_하천방류,일평균이용현황(톤/일)_도로청소,일평균이용현황(톤/일)_공원용수,일평균이용현황(톤/일)_화장실세척,일평균이용현황(톤/일)_건물용수,미사용_하수도방류(톤/일),일평균이용현황(톤/일)_기타건물용수
0,서초오퓨런스빌딩,서초대로 254(서초동 1553-5),18/8,31215.79,2024,6870.5,37.75,NaN,NaN,NaN,NaN,0.0,37.75,NaN
1,더클래스효성(주),효령로55길 65(서초동 1597-7),/,NaN,2024,2318.7,12.74,NaN,NaN,NaN,NaN,0.0,12.74,NaN
2,웅진타워,반포대로30길 81(서초동),16/8,13852.05,2024,21481.5,118.03,NaN,NaN,NaN,NaN,0.0,118.03,NaN
3,나비호텔레지던스(김영*),서초대로58길 5(서초동 1676-2),11/2,2086.08,2024,358.5,1.97,NaN,NaN,NaN,NaN,0.0,1.97,NaN
4,북한산코오롱하늘채아파트,우이천로 367,18/,43216.74,2024,816.1,4.48,0.0,0.0,0.0,0.0,0.0,4.48,NaN


In [121]:
building_underground_water_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18206 entries, 0 to 18205
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   건축물명                 18201 non-null  object 
 1   건축물 위치               18008 non-null  object 
 2   층수(지상_지하)            15551 non-null  object 
 3   연면적                  3699 non-null   object 
 4   조사년도                 18206 non-null  int64  
 5   총발생량(톤)              17613 non-null  float64
 6   일평균발생량(톤/일)          18113 non-null  float64
 7   일평균이용현황(톤/일)_하천방류    1996 non-null   float64
 8   일평균이용현황(톤/일)_도로청소    1943 non-null   float64
 9   일평균이용현황(톤/일)_공원용수    3252 non-null   float64
 10  일평균이용현황(톤/일)_화장실세척   2788 non-null   float64
 11  일평균이용현황(톤/일)_건물용수    7800 non-null   float64
 12  미사용_하수도방류(톤/일)       17936 non-null  float64
 13  일평균이용현황(톤/일)_기타건물용수  396 non-null    object 
dtypes: float64(8), int64(1), object(5)
memory usage: 1.9+ MB


In [122]:
building_underground_water_df.isnull().sum()

건축물명                       5
건축물 위치                   198
층수(지상_지하)               2655
연면적                    14507
조사년도                       0
총발생량(톤)                  593
일평균발생량(톤/일)               93
일평균이용현황(톤/일)_하천방류      16210
일평균이용현황(톤/일)_도로청소      16263
일평균이용현황(톤/일)_공원용수      14954
일평균이용현황(톤/일)_화장실세척     15418
일평균이용현황(톤/일)_건물용수      10406
미사용_하수도방류(톤/일)           270
일평균이용현황(톤/일)_기타건물용수    17810
dtype: int64

In [123]:
# 일평균이용현황(톤/일)_하천방류, 도로청소, 공원용수, 화장실세척, 건물용수, 기타건물용수 컬럼은 결측치가 너무 많아서 사용하기 어려워 보임
building_underground_water_df.drop(columns=["일평균이용현황(톤/일)_하천방류", "일평균이용현황(톤/일)_도로청소", "일평균이용현황(톤/일)_공원용수", "일평균이용현황(톤/일)_화장실세척", "일평균이용현황(톤/일)_건물용수", "일평균이용현황(톤/일)_기타건물용수"], inplace=True)

In [124]:
# 연면적은 사용할 수도 있는데 결측치가 너무 많아서 제거 따로 구할 수 있는 방법도 없음
building_underground_water_df.drop(columns=["연면적"], inplace=True)

In [125]:
building_underground_water_df.isnull().sum()

건축물명                 5
건축물 위치             198
층수(지상_지하)         2655
조사년도                 0
총발생량(톤)            593
일평균발생량(톤/일)         93
미사용_하수도방류(톤/일)     270
dtype: int64

In [126]:
building_underground_water_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18206 entries, 0 to 18205
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   건축물명            18201 non-null  object 
 1   건축물 위치          18008 non-null  object 
 2   층수(지상_지하)       15551 non-null  object 
 3   조사년도            18206 non-null  int64  
 4   총발생량(톤)         17613 non-null  float64
 5   일평균발생량(톤/일)     18113 non-null  float64
 6   미사용_하수도방류(톤/일)  17936 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 995.8+ KB


In [127]:
# "총발생량(톤)", "일평균발생량(톤/일)", "미사용_하수도방류(톤/일)" 이거 3개 모두가 결측치인 행은 지우기 (정보가 없음)
null_rows = building_underground_water_df[
    building_underground_water_df[["총발생량(톤)", "일평균발생량(톤/일)", "미사용_하수도방류(톤/일)"]].isnull().all(axis=1)
]

print(f"결측치만 있는 행 수: {len(null_rows)}")

building_underground_water_df = building_underground_water_df.drop(null_rows.index)

결측치만 있는 행 수: 7


In [128]:
# 건축물명, 건축물 위치가 결측치인 행 지우기
building_underground_water_df.dropna(subset=["건축물명", "건축물 위치"], inplace=True)

In [129]:
building_underground_water_df.isnull().sum()

건축물명                 0
건축물 위치               0
층수(지상_지하)         2576
조사년도                 0
총발생량(톤)            586
일평균발생량(톤/일)         86
미사용_하수도방류(톤/일)     263
dtype: int64

In [130]:
building_underground_water_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17996 entries, 0 to 18205
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   건축물명            17996 non-null  object 
 1   건축물 위치          17996 non-null  object 
 2   층수(지상_지하)       15420 non-null  object 
 3   조사년도            17996 non-null  int64  
 4   총발생량(톤)         17410 non-null  float64
 5   일평균발생량(톤/일)     17910 non-null  float64
 6   미사용_하수도방류(톤/일)  17733 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.1+ MB


In [131]:
building_underground_water_df.to_csv("data/processed/building_underground_water_processed.csv", index=False, encoding="utf-8-sig")